In [12]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree as KDTree
from scipy.stats import gaussian_kde
import sys
import time
import cv2

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")

In [4]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

In [5]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*4 # encoder dim
h_P_dim = 512*4 # decoder dim

lr = 1e-4 # learning rate

# problem dimenc_dimsions
nPreviousStates = 1 # number of previous states to condition on the steering prediction
dim = 4 # (x, y, yaw, steer)

z_dim = 1 # latent dim
y_dim = dim # renp.piruction of the original point (unsused?)

nDrawnSamples = 10 # number of dependent samples to draw during smapling (length of predicted control series)
dataElements = (nDrawnSamples + nPreviousStates) * dim + dim - 1

X_dim = nDrawnSamples*dim # steering
c_dim = dataElements - X_dim # dimension of conditioning variable

network_short = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/networks/stateAndCtrl_park_nonscaled_10short.pt')

nDrawnSamples = 20 # number of dependent samples to draw during smapling (length of predicted control series)
dataElements = (nDrawnSamples + nPreviousStates) * dim + dim - 1

X_dim = nDrawnSamples*dim # steering
c_dim = dataElements - X_dim # dimension of conditioning variable

network_long = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/networks/stateAndCtrl_park_nonscaled_20long.pt')

In [7]:
# load map file

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/motion_planning/maps/map_parking.png'
mapImg = cv2.imread(filename, 0)
blurImg = cv2.blur(mapImg, ksize=(3,3))
occGrid = np.clip(mapImg, 0, 1)
inflatedGrid = np.floor(blurImg / 255)

assert occGrid.shape[0] == occGrid.shape[1]
gridSize = occGrid.shape[0]

In [9]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/motion_planning/data/ctrlDataPfollowPark_18000plans'
rawdata = np.genfromtxt(filename, delimiter=',', dtype='d', usecols=[0,1,2,3,7,8,9]) # disregard init

_, pathsIdx = np.unique(rawdata[:,4:], axis=0, return_index=True)
pathsIdx.sort()

pathsLengths = np.roll(pathsIdx, -1) - pathsIdx
pathsLengths[-1] = rawdata.shape[0] - np.sum(pathsLengths[:-1])

validLengthsIdx = np.argwhere(pathsLengths >= nPreviousStates + nDrawnSamples) 
validPlansIdx = pathsIdx[validLengthsIdx]

data = np.zeros(shape=(1, dataElements)) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
tempdata = np.copy(data) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
for ci, i in enumerate(validPlansIdx.reshape(-1)):
    #rawdata[i:i+pathsLenghts[ci], dim-1] = uniform_filter1d(rawdata[i:i+pathsLenghts[ci], dim-1], size=1, origin=0, mode='nearest', axis=0) # average over steering
    for j in range(pathsLengths[ci] - (nPreviousStates + nDrawnSamples - 1)):
        sample = np.arange(start=j, stop=j + nPreviousStates + nDrawnSamples)

        if np.any(np.abs(np.diff(rawdata[i + sample, 0])) > 0.05) or np.any(np.abs(np.diff(rawdata[i + sample, 1])) > 0.05):
            break # ignore duplicate plans (data generation is not perfect)

        tempdata = np.vstack((tempdata, np.append(rawdata[i + sample, :dim].reshape(1, (nPreviousStates + nDrawnSamples)*dim), rawdata[i, dim:].reshape(1, dim-1), axis=1))) 
        if ci % 500 == 0:
            data = np.vstack((data, tempdata[1:]))
            tempdata = np.zeros(shape=(1, dataElements))

numEntries = data.shape[0]
print(data.shape)

(84250, 87)


In [10]:
# generate conditioning variable

nextStates = data[:, nPreviousStates*dim:-dim+1]
goalState = data[:, -dim+1:]

prevStatesConditions = data[:, :nPreviousStates * dim]
goalStateCondition = data[:, -dim+1:]

cs = np.concatenate((prevStatesConditions, goalStateCondition), axis=1)

print(cs.shape)

(84250, 7)


In [144]:
# define planning problem

def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj


class MyNNControlSampler(oc.ControlSampler):
    def __init__(self, cspace):
        super(MyNNControlSampler, self).__init__(cspace)
        self.name = "my sampler"
        self.cbounds = cspace.getBounds()

    def sampleNext(self, control, previous, state):
        global nDrawnStates
        nDrawnStates += 1

        dist2goal = np.linalg.norm([state.getX()-goal[0], state.getY() - goal[1]])

        if np.random.uniform() < p and dist2goal > 0.2:
            numSamples = 100
            currentState = np.array([state.getX(), state.getY(), state.getYaw(), previous[0]])
            steerAngles = np.linspace(self.cbounds.low[0], self.cbounds.high[0], num=100)

            c = torch.from_numpy(np.repeat([np.append(currentState, goal)],numSamples,axis=0)).float().to(DEVICE)

            if dist2goal > 0.42:
                predictedStates = network_long(torch.randn(numSamples, z_dim).to(DEVICE), c, encode=False).cpu().detach().numpy()
            else:
                predictedStates = network_short(torch.randn(numSamples, z_dim).to(DEVICE), c, encode=False).cpu().detach().numpy()


            kdes = np.array([gaussian_kde(predictedStates[:, i * dim - 1]) for i in range(1, 2)])
            delta = float(np.mean(steerAngles[[np.argmax(kde.pdf(steerAngles)) for kde in kdes]]))

            control[0] = delta + np.random.uniform(low=-0.01, high=0.01)

            # if np.random.uniform() < 0.001:
            #     plt.scatter(predictedStates[:,::dim] * gridSize, predictedStates[:,1::dim] * gridSize, color='green', s=70, alpha=0.1) # nn samples
            #     plt.quiver(currentState[0] * gridSize, currentState[1] * gridSize, np.cos(currentState[2]), np.sin(currentState[2]), color='red', scale=8.0, width=0.015)
            #     plt.quiver(goal[0] * gridSize, goal[1] * gridSize, np. cos(goal[2]), np.sin(goal[2]), color='blue', scale=8.0, width=0.01)
            #     plt.imshow(mapImg, extent=[0, gridSize, 0, gridSize], cmap='gray')
            #     plt.show()
        else:
            control[0] = float(np.random.uniform(low=self.cbounds.low[0], high=self.cbounds.high[0]))


class MyUniformControlSampler(oc.ControlSampler):
    def __init__(self, cspace):
        super(MyUniformControlSampler, self).__init__(cspace)
        self.name = "my sampler"
        self.cbounds = cspace.getBounds()

    def sample(self, control):
        global nDrawnStates
        nDrawnStates += 1

        control[0] = float(np.random.uniform(low=self.cbounds.low[0], high=self.cbounds.high[0]))


def allocNNControlSampler(cspace):
    return MyNNControlSampler(cspace)


def allocUniformControlSampler(cspace):
    return MyUniformControlSampler(cspace)


def isStateValid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied
    u = int(np.floor(state.getX() * occGrid.shape[1])) # right pointing image axis
    v = int(np.floor(occGrid.shape[0] * (1 - state.getY()))) # down pointing image axis

    if spaceInformation.satisfiesBounds(state):
        return inflatedGrid[v,u] > 0


def propagate(start, control, duration, state):
    state.setX(start.getX() + v * duration * np.cos(start.getYaw()))
    state.setY(start.getY() + v * duration * np.sin(start.getYaw()))
    state.setYaw(start.getYaw() + v / L * np.tan(control[0]) * duration)


def problemDef(useNN=False):
    # construct the state space we are planning in
    space = ob.SE2StateSpace()

    # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(2)
    bounds.setLow(0.001)
    bounds.setHigh(0.999)
    space.setBounds(bounds)

    # create a control space
    cspace = oc.RealVectorControlSpace(space, 1)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(1)
    cbounds.setLow(-1.3)
    cbounds.setHigh(1.3)
    cspace.setBounds(cbounds)

    ss = oc.SimpleSetup(cspace)
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))
    # set state sampler
    if useNN:
        cspace.setControlSamplerAllocator(oc.ControlSamplerAllocator(allocNNControlSampler))
    else: 
        cspace.setControlSamplerAllocator(oc.ControlSamplerAllocator(allocUniformControlSampler))

    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isStateValid, ss.getSpaceInformation())))

    si = ss.getSpaceInformation()
    si.setPropagationStepSize(.4)
    si.setMaxControlDuration(1)
    si.setMinControlDuration(1)
    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))

    planner = oc.RRT(si)
    ss.setPlanner(planner)

    return ss

def plan(planObj, initState, goalState):

    space = ob.SE2StateSpace()
    
    start = ob.State(space)
    start().setX(initState[0])
    start().setY(initState[1])
    start().setYaw(initState[2])

    goal = ob.State(space)
    goal().setX(goalState[0])
    goal().setY(goalState[1])
    goal().setYaw(goalState[2])
    
    planObj.setStartAndGoalStates(start, goal, 0.1)

    timeTerminationCondition = ob.timedPlannerTerminationCondition(20.0)
    exactSolTerminationCondition = ob.exactSolnPlannerTerminationCondition(planObj.getProblemDefinition())
    costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), epsilon=0.02)

    solved = planObj.solve(ob.plannerOrTerminationCondition(costTerminationCondition, timeTerminationCondition))

In [ ]:
# compare performance of planning w/ and w/out the learned distributions

nTests = 10

p = 0.8 # likelihood which to sample from NN samples (when useNN is True)
v = 0.1
L = 0.2 # vehicle length constant
minCostThreshold = 0.0 # set minimumcost threshold to only attempt complex planning scenarios

pdefs = [problemDef(useNN=True), problemDef(useNN=False)]

nIterations = np.zeros(shape=(2, nTests))

nCompletedTests = 0

plotPaths = True
while nCompletedTests <= nTests - 1:

    planIdx = np.random.randint(0,numEntries-1); # chose a random test scenario

    start = data[planIdx, :dim-1] # x, y, yaw
    goal = data[planIdx, -dim+1:]

    for i, pdef in enumerate(pdefs):
        pdef.clear()
        nDrawnStates = 0
        plan(pdef, initState=start, goalState=goal)

        if pdef.getProblemDefinition().hasExactSolution():

            sol = np.loadtxt(StringIO(pdef.getProblemDefinition().getSolutionPath().printAsMatrix()))
            nIterations[i, nCompletedTests] = nDrawnStates

            if plotPaths:
                plt.imshow(occGrid, extent=[0, gridSize-1, 0, gridSize-1], cmap='gray')
                plt.quiver(sol[:, 0] * gridSize, sol[:, 1] * gridSize, np.cos(sol[:,2]), np.sin(sol[:,2]), color="purple", scale=8.0, width=0.015, alpha=0.9)
                plt.quiver(start[0] * gridSize, start[1] * gridSize, np.cos(start[2]), np.sin(start[2]), color="red", scale=8.0, width=0.015) # init
                plt.quiver(goal[0] * gridSize, goal[1] * gridSize, np.cos(goal[2]), np.sin(goal[2]), color="blue", scale=8.0, width=0.015) # goal
                plt.show()

        else:
            nCompletedTests -= 1
            break
    
    nCompletedTests += 1


In [158]:
print(nIterations)
print('Mean num iterations: {}, std: {}'.format(np.mean(nIterations, axis=1), np.std(nIterations, axis=1)))
# 10 tests: Mean num iterations: [1566.3 9781.2], std: [1130.6143507  7092.84436316]


[[  707.  1115.  3037.  1152.  3832.  1045.  1347.   660.  2642.   126.]
 [ 5009. 11174.  6779. 13336. 24661.   886. 13053.  2306. 16977.  3631.]]
Mean num iterations: [1566.3 9781.2], std: [1130.6143507  7092.84436316]
